In [1]:
import networkx as nx
from itertools import combinations
import os
import re
import regex
import json

In [2]:
loaded_graph = True

In [3]:
def save_graph():
    nx.write_gml(G, 'graph_index.gml')

In [4]:
def load_graph():
    try:
        return nx.read_gml('graph_index.gml')
    except FileNotFoundError or nx.NetworkXError:
        loaded_graph = False
        return nx.Graph()

In [5]:
G = load_graph()

In [6]:
# companies Blato s.r.o
all_companies = {} # key filename, value companies[]
if not loaded_graph:
    for base_path_n in range(1, 7):
        companies_pase_path = f'/usr/src/HTML/companies_output_part_{base_path_n}/'
        companies_collection = iter(sorted(os.listdir(companies_pase_path)))
        for current_file_path in companies_collection:
            if '_SUCCESS' not in current_file_path and not str(current_file_path).startswith('.'):
                with open(companies_pase_path + current_file_path, 'r', encoding='utf-8') as json_file:
                    data = json.loads(json_file.read())
                    companies = data['entity']
                    if companies:
                        all_companies[data['file_name']] = companies

In [7]:
# people with companies
if not loaded_graph:
    for base_path_n in range(1, 7):
        base_path = f'/usr/src//HTML/entity_names_output_part_{base_path_n}/'
        collection = iter(sorted(os.listdir(base_path)))
        for current_file_path in collection:
            if '_SUCCESS' not in current_file_path and not str(current_file_path).startswith('.'):
                try:
                    with open(base_path + current_file_path, 'r', encoding='utf-8') as json_file:
                        data = json.loads(json_file.read())
                        entities = data['entity']
                        if entities:
                            companies = all_companies.get(data['file_name'], None)
                            if companies:
                                entities.extend(list(set(companies)))
                        if len(entities) > 1:
                            for edge in combinations(set(entities), 2):
                                if len(edge) == 2:
                                    if G.has_edge(*edge): # add weight
                                        G[edge[0]][edge[1]]['weight'] += 1
                                        G[edge[1]][edge[0]]['weight'] += 1
                                    else:
                                        G.add_edge(edge[0], edge[1], weight = 1)
                                        G.add_edge(edge[1], edge[0], weight = 1)
                except Exception as e:
                    continue

In [8]:
if not loaded_graph:
    save_graph()

In [9]:
def get_connections_if_any(query):
    any_connections = re.search('connections:.*;', query)
    connections_entities_to_find = []
    if any_connections:
        any_connections = any_connections.group(
            0).replace('connections:', '')
        entities = regex.findall('([a-zA-Z \.]*)', any_connections)
        connections_entities_to_find = [
            entity.strip() for entity in entities if entity != '']
    return connections_entities_to_find

In [17]:
neighbors = set()
# connections_entities = get_connections_if_any('connections: Andrej Kiska;')
# connections_entities = get_connections_if_any('connections: Blato s.r.o;')
# connections_entities = get_connections_if_any('connections: Robert Fico;')
connections_entities = get_connections_if_any('connections: Robert Fico; Peter Weiss;')
for entity in connections_entities:
    try:
        neighbors.update(x[0] for x in sorted(
            G[entity].items(), key=lambda edge: edge[1]['weight']))
    except KeyError:
        pass
if neighbors:
    print('\n', connections_entities, ': ', ', '.join(
        list(neighbors - set(connections_entities))[:10]))


 ['Robert Fico', 'Peter Weiss'] :  Juraj Kapinaj, Ondrej Urban, Jan Werich, Miroslava Koll, Pavol Pavlis, Lenka Ivanov, Gabriela Gregorovi, Ladislav Kondrc, Marek Domin, Michal Horv
